In [1]:
import time
import requests
import gspread
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from pyNBA.Data.helpers import Helpers
from pyNBA.Data.constants import CURRENT_SEASON, UNKNOWN_PLAYERS
from nba_api.stats.endpoints import CommonTeamRoster
from nba_api.stats.static.teams import find_team_by_abbreviation

import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
helpers = Helpers()

team_to_opp_team = {}
team_to_status = {}

current_player_data = pd.DataFrame(columns=[
    'TEAM', 'NAME', 'START', 'PLAYERSTATUS', 'PLAYERCHANCE'
])

URL = 'https://www.rotowire.com/basketball/nba-lineups.php'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

games = soup.find_all('div', class_='lineup is-nba')
for game in games:
    away_team = game.find('a', class_='lineup__team is-visit').find('div', class_='lineup__abbr').text
    away_team = helpers.prepare_team(away_team)
    away_lineup = game.find('ul', class_='lineup__list is-visit')
    away_player_data, away_lineup_status = helpers.get_player_data(away_lineup)
    away_player_data['TEAM'] = away_team
    away_player_data['NAME'] = away_player_data['NAME'].apply(lambda x: helpers.prepare_name(x, away_team))

    home_team = game.find('a', class_='lineup__team is-home').find('div', class_='lineup__abbr').text
    home_team = helpers.prepare_team(home_team)
    home_lineup = game.find('ul', class_='lineup__list is-home')
    home_player_data, home_lineup_status = helpers.get_player_data(home_lineup)
    home_player_data['TEAM'] = home_team
    home_player_data['NAME'] = home_player_data['NAME'].apply(lambda x: helpers.prepare_name(x, home_team))

    team_to_opp_team[away_team] = home_team
    team_to_opp_team[home_team] = away_team
    team_to_status[away_team] = away_lineup_status
    team_to_status[home_team] = home_lineup_status

    current_player_data = current_player_data.append(away_player_data)
    current_player_data = current_player_data.append(home_player_data)

current_player_data = current_player_data.loc[
    ~current_player_data['NAME'].isin(UNKNOWN_PLAYERS)
]

roster_data = pd.DataFrame()
for team_abbreviation in current_player_data['TEAM'].unique():
    print(team_abbreviation)
    team = find_team_by_abbreviation(team_abbreviation)
    team_id = team['id']

    roster = CommonTeamRoster(season=CURRENT_SEASON, team_id=team_id).get_data_frames()[0]
    time.sleep(0.500)

    roster['TEAM'] = team['abbreviation']

    roster_data = roster_data.append(roster)

roster_data = roster_data.rename(columns={'TeamID': 'TEAMID', 'PLAYER_ID': 'PLAYERID', 'PLAYER': 'NAME'})

roster_data['POSITION'] = roster_data['POSITION'].str.replace('G', 'Guard')
roster_data['POSITION'] = roster_data['POSITION'].str.replace('F', 'Forward')
roster_data['POSITION'] = roster_data['POSITION'].str.replace('C', 'Center')

current_data = roster_data.merge(current_player_data, on=['NAME', 'TEAM'], how='left')

current_data['LINEUPSTATUS'] = current_data['TEAM'].apply(lambda x: team_to_status[x])
current_data['OPP_TEAM'] = current_data['TEAM'].apply(lambda x: team_to_opp_team[x])

current_data['START'] = current_data['START'].fillna(0)
current_data['PLAYERSTATUS'] = current_data['PLAYERSTATUS'].fillna('Healthy')
current_data['PLAYERCHANCE'] = current_data['PLAYERCHANCE'].fillna(100)
current_data['SEASON'] = CURRENT_SEASON

current_data = current_data[[
    'SEASON', 'LINEUPSTATUS', 'PLAYERID', 'TEAM', 'OPP_TEAM', 'NAME', 'POSITION',
    'START', 'PLAYERSTATUS', 'PLAYERCHANCE'
    ]]

IND
ORL
MIN
BOS
MEM
NYK
CHI
ATL
PHI
NOP
SAS
DEN
CHA
MIL
WAS
GSW
HOU
LAC


In [3]:
missing_players = current_player_data.loc[
    ~current_player_data['NAME'].isin(roster_data['NAME'].unique())
    ]['NAME'].unique()

In [4]:
print(missing_players)

['Shaquille Harrison' 'J. Bell' 'G. Payton']


In [5]:
for i in missing_players:
    last = i.split()[1]
    print('------------------------------------------------------------------------------------------------------------------------')
    print(last)
    display(roster_data.loc[roster_data['NAME'].str.contains(last)])
    print('------------------------------------------------------------------------------------------------------------------------')

------------------------------------------------------------------------------------------------------------------------
Harrison


,TEAMID,SEASON,LeagueID,NAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYERID,TEAM


------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
Bell


,TEAMID,SEASON,LeagueID,NAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYERID,TEAM


------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
Payton


,TEAMID,SEASON,LeagueID,NAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYERID,TEAM
4,1610612738,2020,00,Payton Pritchard,payton-pritchard,11,Guard,6-1,195,"JAN 28, 1998",23.0,R,Oregon,1630202,BOS
5,1610612752,2020,00,Elfrid Payton,elfrid-payton,6,Guard,6-3,195,"FEB 22, 1994",27.0,6,Louisana-Lafayette,203901,NYK
9,1610612744,2020,00,Gary Payton II,gary-payton-ii,20,Guard,6-3,195,"DEC 01, 1992",28.0,4,Oregon State,1627780,GSW


------------------------------------------------------------------------------------------------------------------------
